In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

# 加载数据
iris = load_iris()
X = iris.data
y = iris.target

# 标准化特征
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 转换为Tensor
X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)

y_train = torch.tensor(y_train).long()
y_test = torch.tensor(y_test).long()

#y_train = one_hot(torch.tensor(y_train, dtype=int))
#y_test = one_hot(torch.tensor(y_test, dtype=int))
print(y_train)
y_train.shape

tensor([0, 0, 1, 0, 0, 2, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 2, 2, 1, 2, 1, 2, 1, 0,
        2, 1, 0, 0, 0, 1, 2, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 1, 1, 2, 1,
        0, 1, 2, 0, 0, 1, 1, 0, 2, 0, 0, 1, 1, 2, 1, 2, 2, 1, 0, 0, 2, 2, 0, 0,
        0, 1, 2, 0, 2, 2, 0, 1, 1, 2, 1, 2, 0, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 1,
        2, 2, 0, 1, 2, 2, 0, 2, 0, 1, 2, 2, 1, 2, 1, 1, 2, 2, 0, 1, 2, 0, 1, 2])


torch.Size([120])

In [74]:
# 定义数据集类
class IrisDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index], self.targets[index]
    
# 定义模型
class RBF(nn.Module):
    def __init__(self, in_features, out_features):
        super(RBF, self).__init__()
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        return torch.exp(-torch.norm(x - self.linear.weight, dim=1) ** 2).unsqueeze(1)

class MultiKernelNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MultiKernelNet, self).__init__()
        #self.layer1 = nn.Linear(input_dim, hidden_dim)
        self.rbf = RBF(input_dim, hidden_dim) # RBF kernel
        self.linear = nn.Linear(hidden_dim, hidden_dim) # linear kernel
        self.rbf2 = RBF(hidden_dim, hidden_dim) # RBF kernel
        self.layer2 = nn.Linear(hidden_dim, output_dim)
        # convert to (batch_size, class_num)
        
    def forward(self, x):
        # 第一层：使用RBF核函数
        rbf_output = torch.matmul(self.rbf(x),self.rbf(x).t())
        #print('RBF输出结果{},尺寸为: {}'.format(rbf_output,rbf_output.shape))
        #rbf_output = torch.relu(rbf_output)
        # 第二层：使用线性核函数
        linear_output = self.layer2(torch.matmul(self.linear(rbf_output),self.linear(rbf_output).t()))
        #rbf2_output = self.layer2(torch.matmul(self.rbf2(rbf_output),self.rbf2(rbf_output).t()))
        # 求取最终输出
        output = F.softmax(linear_output, dim=1)
        #output = F.softmax(rbf2_output, dim=1)
        #print('输出结果{},尺寸为: {}'.format(output,output.shape))
        
        return output

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, precision_score, recall_score, average_precision_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix
def resultAnalysis(testData,testLabels,pred,proba,_module=None):
    xmodulelist = ['macro','micro','weighted','non-avg']
    criteion = ['Accuracy','Precision','Recall','F1_score','AUC','AUPRC']
    df = pd.DataFrame(index=xmodulelist,columns=criteion)
    
    
    if _module not in xmodulelist:
        
        df['Accuracy']['non-avg']=round(accuracy_score(testLabels,pred),3)
        df['Precision']['non-avg']=round(precision_score(testLabels,pred,),3)
        df['Recall']['non-avg']=round(recall_score(testLabels,pred,),3)
        df['AUC']['non-avg']=round(roc_auc_score(testLabels,proba,),3)
        df['AUPRC']['non-avg']=round(average_precision_score(testLabels,pred,),3)
        df['F1_score']['non-avg']=round(f1_score(testLabels,pred,),3)
        print(df.iloc[3,:])
            
    else:
        df['Accuracy'][_module]=round(accuracy_score(testLabels,pred),3)
        df['Precision'][_module]=round(precision_score(testLabels,pred,average=_module),3)
        df['Recall'][_module]=round(recall_score(testLabels,pred,average=_module),3)
        df['AUC'][_module]=round(roc_auc_score(testLabels,proba,average=_module),3)
        df['AUPRC'][_module]=round(average_precision_score(testLabels,pred,average=_module),3)
        df['F1_score'][_module]=round(f1_score(testLabels,pred,average=_module),3)
        print(df.loc[_module,:])
    print(confusion_matrix(testLabels,pred))
    return classification_report(testLabels,pred)

In [ ]:
from sklearn import svm

class SVMC(nn.Module):
    def __init__(self, in_features, out_features,k='linear'):
        super(SVMC, self).__init__()
        #self.svmc = nn.Linear(in_features, out_features)
        self.linear = nn.Linear(in_features, out_features)

    def forward(self, x):
        clf_proba = svm.SVC(kernel=k,C=1.0,probability=True).fit(trainData,trainLabels)
        clf_pred = clf_proba.predict(testData)
        result = clf_proba.score(testData, testLabels)
        score = clf_proba.decision_function(testData)

        rocauc = roc_auc_score(testLabels, score)
        recall = recall_score(testLabels, clf_pred)
        precision = precision_score(testLabels, clf_pred)
        print("The kernel {} prediction is:".format(k))
        print(resultAnalysis(testData,testLabels,clf_pred,score,))
        return torch.exp(-torch.norm(x - self.linear.weight, dim=1) ** 2).unsqueeze(1)

In [76]:
# 定义超参数
input_dim = X_train.shape[1]
hidden_dim = 10
output_dim = 3
lr = 0.01
batch_size = 10
epochs = 100

In [77]:
# 定义模型、损失函数和优化器
model = MultiKernelNet(input_dim, hidden_dim, output_dim)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)


# 创建数据加载器
train_dataset = IrisDataset(X_train, y_train)
test_dataset = IrisDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 定义训练和测试函数
def train(model, criterion, optimizer, train_loader):
    model.train()
    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        
        #print('输出结果{},尺寸为: {}'.format(output,output.shape))
        #print('目标对象{},尺寸为: {}'.format(target,target.shape))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    return correct / total


for epoch in range(epochs):
    train(model, criterion, optimizer, train_loader)
    train_acc = test(model, train_loader)
    test_acc = test(model, test_loader)
    print(f"Epoch {epoch+1}: Train accuracy: {train_acc:.4f}, Test accuracy: {test_acc:.4f}")

Epoch 1: Train accuracy: 0.3250, Test accuracy: 0.3667
Epoch 2: Train accuracy: 0.3417, Test accuracy: 0.3000
Epoch 3: Train accuracy: 0.3417, Test accuracy: 0.3000
Epoch 4: Train accuracy: 0.4583, Test accuracy: 0.5333
Epoch 5: Train accuracy: 0.3833, Test accuracy: 0.4667
Epoch 6: Train accuracy: 0.3833, Test accuracy: 0.4667
Epoch 7: Train accuracy: 0.4000, Test accuracy: 0.4667
Epoch 8: Train accuracy: 0.5333, Test accuracy: 0.4000
Epoch 9: Train accuracy: 0.4667, Test accuracy: 0.4667
Epoch 10: Train accuracy: 0.4917, Test accuracy: 0.5000
Epoch 11: Train accuracy: 0.5083, Test accuracy: 0.5000
Epoch 12: Train accuracy: 0.4667, Test accuracy: 0.4667
Epoch 13: Train accuracy: 0.4083, Test accuracy: 0.4667
Epoch 14: Train accuracy: 0.4917, Test accuracy: 0.3667
Epoch 15: Train accuracy: 0.4583, Test accuracy: 0.4000
Epoch 16: Train accuracy: 0.4750, Test accuracy: 0.3667
Epoch 17: Train accuracy: 0.5167, Test accuracy: 0.3667
Epoch 18: Train accuracy: 0.4917, Test accuracy: 0.3667
E

`torch.nn.Linear`是PyTorch中的一个类，用于定义一个线性变换（linear transformation）。

其定义如下：

`class torch.nn.Linear(in_features: int, out_features: int, bias: bool = True)`
其中，in_features是输入特征的数量，out_features是输出特征的数量，bias表示是否包含偏置（bias）项。

`torch.nn.Linear`中包含两个重要的参数：weight和bias。weight是一个大小为(out_features, in_features)的张量，bias是一个大小为(out_features,)的张量。在前向传播的过程中，输入张量会与weight相乘，并加上bias，得到输出张量。

以下是`torch.nn.Linear`的一些性质和用法：

weight和bias可以通过调用model.parameters()方法获取。

在模型训练过程中，PyTorch会自动优化weight和bias，使得模型在训练集上的性能最优。

torch.nn.Linear常常用于构建神经网络中的全连接层。例如，可以将多个torch.nn.Linear组合成一个多层感知机（Multilayer Perceptron，MLP）模型。

torch.nn.Linear可以通过指定in_features和out_features来定义输入和输出的特征数。这对于不同的任务和数据集可以有不同的取值。

torch.nn.Linear还可以设置bias=False，这样就不会在线性变换中加上偏置。

In [34]:
# 导入相关库
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 定义RBF核函数
def rbf_kernel(x):
    # 计算两个张量之间的欧氏距离的平方
    dist = torch.exp(-torch.norm(x - self.linear.weight, dim=1) ** 2)
    # 计算RBF核函数的值
    return torch.sum(dist * dist,dim=1)

# 定义线性核函数
def linear_kernel(x):
    # 计算两个张量之间的点积
    return torch.sum(x * x,dim=1)

# 定义二层多核学习模型
class MultiKernelModel(nn.Module):
    def __init__(self, in_features, out_features, hidden_size):
        super(MultiKernelModel, self).__init__()
        # 第一层使用RBF核函数，参数为隐层神经元的权重
        self.linear1 = nn.Linear(in_features, hidden_size)
        # 第二层使用线性核函数，参数为输出层神经元的权重和偏置
        self.linear2 = nn.Linear(hidden_size, out_features)
        # 使用softmax函数作为激活函数，得到分类概率
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # 计算第一层的输出，使用RBF核函数
        output1 = rbf_kernel(x) #self.linear1.weight
        # 计算第二层的输出，使用线性核函数和偏置
        output2 = linear_kernel(output1) + self.linear2.bias #self.linear2.weight
        # 使用softmax函数得到分类概率
        output = self.softmax(output2)
        return output

# 加载iris数据集，并划分为训练集和测试集
iris = load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 将numpy数组转换为PyTorch张量，并设置为float类型和long类型
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train).float()
y_test = torch.from_numpy(y_test).float()

# 创建模型实例，设置输入特征数为4，输出类别数为3，隐层神经元数为10
model = MultiKernelModel(4, 3, 10)

# 定义交叉熵损失函数
criterion = nn.CrossEntropyLoss()

# 定义SGD优化器，设置学习率为0.01
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 设置训练轮数为100
epochs = 100

# 开始训练过程
for epoch in range(epochs):
    # 清零梯度
    optimizer.zero_grad()
    # 前向传播，得到预测结果
    output = model(X_train)
    # 计算损失值
    loss = criterion(output, y_train)
    # 反向传播，计算梯度
    loss.backward()
    # 更新参数
    optimizer.step()
    # 打印每轮的损失值和准确率
    pred = torch.argmax(output, dim=1)
    acc = accuracy_score(y_train.numpy(), pred.numpy())
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}, Accuracy: {acc:.4f}")


NameError: name 'self' is not defined

一般我们用的R里面的e1071包的svm函数和python的scikit-learn包里的svm函数都是基于libsvm实现的：http://www.csie.ntu.edu.tw/~cjlin/libsvm/libsvm
libsvm提供了下列几个核函数：

linear:u'\*v

polynomial:(gamma*u'*v + coef0)^degree
    
radial basis:exp(-gamma*|u-v|^2)
        
sigmoid:tanh(gamma*u'*v + coef0)
             
通常来说，个人经验RBF kernel是第一选择，几乎效果都是最好的。此外，linear kernel也有用武之地，（其实linear kernel本身就是RBF的一种特殊情况），因为当features的数量很多的时候，linear kernel的速度优势比较明显。当然有时间当然最好是都试一遍，看看哪个更符合自己的数据分布特性了。



In [3]:
model.eval()

with torch.no_grad():
    # 将测试数据移动到GPU上
    X_test, y_test = X_test.to(device), y_test.to(device)
    # 前向传播
    outputs = model(X_test)

    # 预测类别
    _,predicted = torch.max(outputs.data, 1)

    # 计算预测准确率
    total = y_test.size(0)
    correct = (predicted == y_test).sum().item()
    print('Test Accuracy of the model on the {} test images: {:.2f} %'.format(total, 100 * correct / total))


True

In [45]:
print(X_train.size())
print(X_train)
print(y_train.size())
print(y_train)

torch.Size([120, 4])
tensor([[-1.5065e+00,  1.2492e+00, -1.5676e+00, -1.3154e+00],
        [-1.7367e-01,  3.0908e+00, -1.2834e+00, -1.0522e+00],
        [ 1.0380e+00,  9.8217e-02,  3.6490e-01,  2.6414e-01],
        [-1.2642e+00,  7.8881e-01, -1.2266e+00, -1.3154e+00],
        [-1.7489e+00,  3.2841e-01, -1.3971e+00, -1.3154e+00],
        [ 5.5333e-01, -1.2830e+00,  7.0592e-01,  9.2230e-01],
        [ 6.7450e-01,  3.2841e-01,  4.2173e-01,  3.9577e-01],
        [-7.7951e-01,  1.0190e+00, -1.2834e+00, -1.3154e+00],
        [-1.0218e+00,  1.2492e+00, -1.3402e+00, -1.3154e+00],
        [-7.7951e-01,  2.4002e+00, -1.2834e+00, -1.4471e+00],
        [-5.2506e-02, -8.2257e-01,  7.6276e-01,  9.2230e-01],
        [ 1.8983e-01,  7.8881e-01,  4.2173e-01,  5.2741e-01],
        [ 1.0380e+00,  9.8217e-02,  5.3541e-01,  3.9577e-01],
        [-5.3718e-01,  1.9398e+00, -1.3971e+00, -1.0522e+00],
        [-5.3718e-01,  1.4794e+00, -1.2834e+00, -1.3154e+00],
        [-4.1601e-01, -1.5132e+00, -3.2966e-02, -

In [59]:
a = torch.tensor([1, 2, 3, 4, 5])

# 方法1：使用t()方法
a_t = a.t()

# 方法2：使用transpose()方法
a_t2 = a.transpose(0, 0)

#a = torch.tensor([1, 2, 3, 4, 5])
a2 = a.unsqueeze(0)  # 在第0个维度上增加一个维度

torch.matmul(a2.t(),a2)

tensor([[ 1,  2,  3,  4,  5],
        [ 2,  4,  6,  8, 10],
        [ 3,  6,  9, 12, 15],
        [ 4,  8, 12, 16, 20],
        [ 5, 10, 15, 20, 25]])